# requirements.txt

pip install keras==2.2.3 scikit-learn==0.19.0 tensorflow-gpu==1.15.0

pydot  if you have graphdot

source code is from https://github.com/smousavi05/Unsupervised_Deep_Learning

the file is from https://github.com/smousavi05/Unsupervised_Deep_Learning/blob/master/unsupervised_deep_learning.ipynb

In [ ]:
from __future__ import print_function
import numpy as np
import h5py
import glob
import math
import os
import shutil
from scipy import signal
import tensorflow.keras
from scipy.signal import butter, lfilter
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, MaxPooling1D, UpSampling1D, Flatten, Dropout, Reshape 
from keras.layers import Bidirectional, BatchNormalization, ZeroPadding1D, Conv2DTranspose
from keras.models import Model
from keras import backend as K
from keras.optimizers import SGD, Adam
from keras import regularizers
from keras.engine.topology import Layer, InputSpec
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from keras.initializers import VarianceScaling
from keras.callbacks import CSVLogger
from sklearn.utils.linear_assignment_ import linear_assignment
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import idac_metrics as metrics
import matplotlib.pyplot as plt
from matplotlib.pyplot import savefig

import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('muted')

In [ ]:
import logging
logging.disable(30)   # disable the deprecated warning from tensorflow

# Loading the data

In [ ]:
data in original paper

x = np.load('x.np.npy') # STFT of local and teleseismic waveforms
y = np.load('y.np.npy') # labels
n_clusters = 2

the shape is (size, 16, 48, 1)

Next, we will read our data

In [ ]:
from scipy.io import loadmat
train_data = loadmat('earthb.mat')
all_data = train_data['images']
all_target = train_data['labels']

In [ ]:
n_clusters = 3

In [ ]:
from data_utils import get_data


data = get_data(all_data, all_target, dataset='eq', seed=1, shape=[-1, 40, 40, 1], select=[100, 1700], norm=True)


In [ ]:
x_train, x_valid, x_test, y_train, y_valid, y_test, splits, splits_test = data
print('data shape %s' % str(x_train.shape) + str(x_valid.shape) + str(x_test.shape))

# Building the autoencoder

In [ ]:
inp = Input(shape=(40, 40, 1))  
e = Conv2D(4, (3, 3), activation='tanh', padding='same')(inp)
e = MaxPooling2D((2, 2), padding='same')(e)
e = Conv2D(2, (3, 3), activation='tanh', padding='same')(e)
e = MaxPooling2D((2, 2), padding='same')(e)
e = Conv2D(1, (3, 3), activation='tanh', padding='same')(e)
e = MaxPooling2D((2, 2), padding='same')(e)

shape_before_flattening = K.int_shape(e)
encoded = Flatten()(e)
d = Reshape(shape_before_flattening[1:])(encoded)

d = Conv2D(1, (3, 3), activation='tanh', padding='same')(d)
d = UpSampling2D((2, 2))(d)
d = Conv2D(2, (3, 3), activation='tanh', padding='same')(d)
d = UpSampling2D((2, 2))(d)
d = Conv2D(4, (3, 3), activation='tanh', padding='same')(d)
d = UpSampling2D((2, 2))(d)
decoded = Conv2D(1, (3, 3), padding='same')(d)

autoencoder = Model(inputs=inp, outputs=decoded, name='autoencoder')
encoder = Model(inputs=inp, outputs=encoded, name='encoder')
#autoencoder.summary()

# from keras.utils import plot_model
# plot_model(autoencoder, to_file='autoencoder.png', show_shapes=True)
# from IPython.display import Image
# Image(filename='autoencoder.png')

# Pretraining of the autoencoder

In [ ]:
autoencoder.compile(optimizer='adam', loss='mse')
csv_logger = CSVLogger('pretrain_log.csv')

autoencoder.fit(x_train, x_train, batch_size=128, epochs=50, callbacks=[csv_logger])

# Now, adding the clustering layer into the bottelneck layer 

In [ ]:
#### clustering layers
class ClusteringLayer(Layer):
    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight((self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1)) 
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    
print('...Finetuning...')   
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(encoder.output)
model = Model(inputs=encoder.input, outputs=[clustering_layer, autoencoder.output])
model.compile(loss=['kld', 'mse'], loss_weights=[0.1, 1], optimizer='adam')

# from keras.utils import plot_model
# plot_model(model, to_file='model.png', show_shapes=True)
# from IPython.display import Image
# Image(filename='model.png')

# Finetuning of full network

In [ ]:
### initializing the weights using Kmean and assigning them to the model
kmeans = KMeans(n_clusters=n_clusters, n_init=20)
y_pred = kmeans.fit_predict(encoder.predict(x_train))
y_pred_last = np.copy(y_pred)
model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])


In [ ]:
metrics.acc(y_train, y_pred)

In [ ]:

## parameters for the finetuning
batch_size = 128
tol = 0.001 # tolerance threshold to stop training
loss = 0
index = 0
maxiter = 10000
update_interval = int(x_train.shape[0] / batch_size) + 1
index_array = np.arange(x_train.shape[0])

In [ ]:

###############################################################################
### simultaneous optimization and clustering
def target_distribution(q):
    weight = q ** 2 / q.sum(0)
    return (weight.T / weight.sum(1)).T

for ite in range(int(maxiter)):
    if ite % update_interval == 0:
        q, _  = model.predict(x_train, verbose=0)
        p = target_distribution(q)  # update the auxiliary target distribution p       
        y_pred = q.argmax(1) # evaluate the clustering performance

        if y_train is not None:
            acc = np.round(metrics.acc(y_train, y_pred), 5)
            loss = np.round(loss, 5)
            print('Iter %d(Epoch %d): acc = %.5f' % (ite, ite // update_interval, acc), ' ; loss=', loss)

        # check stop criterion
        delta_label = np.sum(y_pred != y_pred_last).astype(np.float32) / y_pred.shape[0]
        y_pred_last = np.copy(y_pred)

        if ite > 0 and delta_label < tol:
            print('delta_label ', delta_label, '< tol ', tol)
            break
        
        IN = encoder.predict(x_train)        
    
    idx = index_array[index * batch_size: min((index+1) * batch_size, x_train.shape[0])]
    loss = model.train_on_batch(x=x_train[idx], y=[p[idx], x_train[idx]])
    index = index + 1 if (index + 1) * batch_size <= x_train.shape[0] else 0  


In [ ]:
enc = encoder.predict(x_train[:3000])   # the data is large, TSNE is slow, so use a small subset
print('Get features, then TSNE')
from sklearn.manifold import TSNE
redu = TSNE(random_state=123).fit_transform(enc)


In [ ]:
def plotter(S, y, target_names):
    '''
    function to visualize the outputs of t-SNE
    '''
    # choose a color palette with seaborn.
    colors = ['navy', 'turquoise', 'darkorange']
    lw = 2
    # create a scatter plot.
    f = plt.figure(figsize=(22, 10))
    ax = f.add_subplot(111)
    for color, i, target_name in zip(colors, [0, 1, 2], target_names):
        plt.scatter(S[y == i, 0], S[y == i, 1], color=color, alpha=.5, lw=lw, s=100, label=target_name)
    plt.legend(loc='lower left', shadow=False, scatterpoints=1, prop={'size': 20})
    ax.axis('off')
    ax.axis('tight') 
    plt.show()

    return f, ax


target_names = ['a', 'b', 'c']
plotter(redu, y_train[:3000], target_names) 